https://github.com/huangtinglin/NGCF-PyTorch/blob/master/NGCF/main.py

In [1]:
import os
import pandas as pd
import numpy as np
import random
import scipy.sparse as sp
import argparse

from sklearn.preprocessing import LabelEncoder

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# 전처리

In [2]:
path = '../Data/Movie_Lens_100k'

In [3]:
train_df = pd.read_csv(os.path.join(path, 'ua.base'), sep = '\t', names = ['user_id', 'movie_id', 'rating', 'timestamp'])
test_df = pd.read_csv(os.path.join(path, 'ua.test'), sep = '\t', names = ['user_id', 'movie_id', 'rating', 'timestamp'])

In [4]:
test_only_movie = list(set(test_df['movie_id'].unique().flatten()) - set(train_df['movie_id'].unique().flatten()))
test_df = test_df[~test_df['movie_id'].isin(test_only_movie)]

### Label Encoder

In [5]:
user_le = LabelEncoder()
movie_le = LabelEncoder()

user_le.fit(train_df['user_id'])
movie_le.fit(train_df['movie_id'])

train_df['user_id'] = user_le.transform(train_df['user_id'])
train_df['movie_id'] = movie_le.transform(train_df['movie_id'])

test_df['user_id'] = user_le.transform(test_df['user_id'])
test_df['movie_id'] = movie_le.transform(test_df['movie_id'])

In [6]:
train_df.head()

,user_id,movie_id,rating,timestamp
0,0,0,5,874965758
1,0,1,3,876893171
2,0,2,4,878542960
3,0,3,3,876893119
4,0,4,3,889751712


# NGCF

### Graph class

In [8]:
class Graph(object):
    def __init__(self, train_df, test_df, user_le, item_le, batch_size):
#         self.path = path
        self.batch_size = batch_size
       
        self.user_le = user_le
        self.item_le = item_le
        
        self.n_users = len(train_df['user_id'].unique())
        self.n_items = len(train_df['movie_id'].unique())
        
        self.n_train = len(train_df)
        self.n_test = len(test_df)
        
        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype = np.float32)
        
        self.train_items, self.test_set = {}, {}
        self.exist_users = list(train_df['user_id'].unique())
        
        for i, row in train_df.iterrows():
            
            # 여기에 rating 값을 넣어야 하는지 생갈할 필요가 있다.
            self.R[row['user_id'], row['movie_id']] = 1
        
        for i, row in enumerate(train_df.groupby(['user_id'])['movie_id'].unique()):
            self.train_items[i] = row
            
        for i, row in enumerate(test_df.groupby(['user_id'])['movie_id'].unique()):
            self.test_set[i] = row
        
    def get_adj_mat(self):
        adj_mat, norm_adj_mat, mean_adj_mat = self.create_adj_mat()
        return adj_mat, norm_adj_mat, mean_adj_mat
    
    def create_adj_mat(self):
        adj_mat = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype = np.float32)
        adj_mat = adj_mat.tolil()
        R = self.R.tolil()
        
        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T
        adj_mat = adj_mat.todok()
        
        def mean_adj_single(adj):
            rowsum = np.array(adj.sum(1))
            
            d_inv = np.power(rowsum, -1).flatten()
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)
            
            norm_adj = d_mat_inv.dot(adj)
            return norm_adj.tocoo()
        
        def normalized_adj_single(adj):
            rowsum = np.array(adj.sum(1))
            
            d_inv_sqrt = np.power(rowsum, -0.5).flatten()
            d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
            d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
            
            bi_lap = d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt)
            return bi_lap.tocoo()
        
        def check_adj_if_equal(adj):
            dense_A = np.array(adj.todense())
            degree = np.sum(dense_A, axis = 1, keepdims = False)
            
            temp = np.dot(np.diag(np.power(degree, -1)), dense_A)
            return temp
        
        norm_adj_mat = mean_adj_single(adj_mat + sp.eye(adj_mat.shape[0]))
        mean_adj_mat = mean_adj_single(adj_mat)
        return adj_mat.tocsr(), norm_adj_mat.tocsr(), mean_adj_mat.tocsr()
    
    
    def get_num_users_items(self):
        return self.n_users, self.n_items        

In [9]:
class CustomDataset(Dataset):
    def __init__(self, df):
        super(Dataset, self).__init__()
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        user = torch.tensor(row[0], dtype = torch.long)
        item = torch.tensor(row[1], dtype = torch.long)
        label = torch.tensor(row[2], dtype = torch.float)
        
        return user, item, label

# Main

### argparse

In [10]:
parser = argparse.ArgumentParser(description="Run NGCF.")
parser.add_argument('--weights_path', nargs='?', default='model/',
                    help='Store model path.')
parser.add_argument('--data_path', nargs='?', default='../Data/',
                    help='Input data path.')
parser.add_argument('--proj_path', nargs='?', default='',
                    help='Project path.')

parser.add_argument('--dataset', nargs='?', default='gowalla',
                    help='Choose a dataset from {gowalla, yelp2018, amazon-book}')
parser.add_argument('--pretrain', type=int, default=0,
                    help='0: No pretrain, -1: Pretrain with the learned embeddings, 1:Pretrain with stored models.')
parser.add_argument('--verbose', type=int, default=1,
                    help='Interval of evaluation.')
parser.add_argument('--epoch', type=int, default=30,
                    help='Number of epoch.')

parser.add_argument('--embed_size', type=int, default=64,
                    help='Embedding size.')
parser.add_argument('--layer_size', nargs='?', default='[64,64,64]',
                    help='Output sizes of every layer')
parser.add_argument('--batch_size', type=int, default=1024,
                    help='Batch size.')

parser.add_argument('--regs', nargs='?', default='[1e-5]',
                    help='Regularizations.')
parser.add_argument('--lr', type=float, default=0.001,
                    help='Learning rate.')

parser.add_argument('--model_type', nargs='?', default='ngcf',
                    help='Specify the name of model (ngcf).')
parser.add_argument('--adj_type', nargs='?', default='norm',
                    help='Specify the type of the adjacency (laplacian) matrix from {plain, norm, mean}.')

parser.add_argument('--gpu_id', type=int, default=6)

parser.add_argument('--node_dropout_flag', type=int, default=1,
                    help='0: Disable node dropout, 1: Activate node dropout')
parser.add_argument('--node_dropout', nargs='?', default='[0.1]',
                    help='Keep probability w.r.t. node dropout (i.e., 1-dropout_ratio) for each deep layer. 1: no dropout.')
parser.add_argument('--mess_dropout', nargs='?', default='[0.1,0.1,0.1]',
                    help='Keep probability w.r.t. message dropout (i.e., 1-dropout_ratio) for each deep layer. 1: no dropout.')

parser.add_argument('--Ks', nargs='?', default='[20, 40, 60, 80, 100]',
                    help='Output sizes of every layer')

parser.add_argument('--save_flag', type=int, default=0,
                    help='0: Disable model saver, 1: Activate model saver')

parser.add_argument('--test_flag', nargs='?', default='part',
                    help='Specify the test type from {part, full}, indicating whether the reference is done in mini-batch')

parser.add_argument('--report', type=int, default=0,
                    help='0: Disable performance report w.r.t. sparsity levels, 1: Show performance report w.r.t. sparsity levels')

_StoreAction(option_strings=['--report'], dest='report', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help='0: Disable performance report w.r.t. sparsity levels, 1: Show performance report w.r.t. sparsity levels', metavar=None)

In [17]:
args = parser.parse_args('')
args.device = torch.device('cpu')

data_generator = Graph(train_df, test_df, user_le, movie_le, args.batch_size)

plain_adj, norm_adj, mean_adj = data_generator.get_adj_mat()

args.node_dropout = eval(args.node_dropout)
args.mess_dropout = eval(args.mess_dropout)


model = NGCF(data_generator.n_users,
            data_generator.n_items,
            norm_adj,
            args).to(args.device)


optimizer = optim.Adam(model.parameters(), lr = args.lr)

train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

train_dataloader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = args.batch_size, shuffle = True)

train_loss_loger, test_loss_loger = [], []

for epoch in range(args.epoch):
    train_loss = 0
    
    model.train()
    for users, items, labels in train_dataloader:
        users_embedding, items_embedding = model(users, items)
                
        batch_loss = model.loss(users_embedding, items_embedding, labels)
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss.item()/len(train_dataloader)
    
    train_loss_loger.append(train_loss)
    
    test_loss = 0
    with torch.no_grad():
        model.eval()
        for users, items, labels in test_dataloader:
            users_embedding, items_embedding = model(users, items)
            
            batch_loss = model.loss(users_embedding, items_embedding, labels)
            test_loss += batch_loss.item() / len(test_dataloader)

        test_loss_loger.append(test_loss)
        
    print('epoch : {}, train_loss : {}, test_loss : {}'.format(epoch, round(train_loss, 4), round(test_loss, 4)))

epoch : 0, train_loss : 1.9686, test_loss : 2.0468
epoch : 1, train_loss : 1.8886, test_loss : 1.9195
epoch : 2, train_loss : 1.4726, test_loss : 1.4204
epoch : 3, train_loss : 1.0101, test_loss : 1.1402
epoch : 4, train_loss : 0.888, test_loss : 1.0519
epoch : 5, train_loss : 0.8325, test_loss : 0.9854
epoch : 6, train_loss : 0.7776, test_loss : 0.9617
epoch : 7, train_loss : 0.7288, test_loss : 0.9197
epoch : 8, train_loss : 0.6922, test_loss : 0.9114
epoch : 9, train_loss : 0.6566, test_loss : 0.9095
epoch : 10, train_loss : 0.6236, test_loss : 0.9107
epoch : 11, train_loss : 0.592, test_loss : 0.8742
epoch : 12, train_loss : 0.5619, test_loss : 0.89
epoch : 13, train_loss : 0.5344, test_loss : 0.8938
epoch : 14, train_loss : 0.5072, test_loss : 0.8846
epoch : 15, train_loss : 0.4822, test_loss : 0.8884
epoch : 16, train_loss : 0.4587, test_loss : 0.9013
epoch : 17, train_loss : 0.4362, test_loss : 0.8893
epoch : 18, train_loss : 0.4148, test_loss : 0.9068
epoch : 19, train_loss : 0